In [10]:
import numpy as np
import pandas as pd
import sklearn

In [11]:
# import dataset:
gleam = pd.read_csv("GLEAM_LivestockEmissions.csv")
gleamDF = pd.DataFrame(gleam)

In [12]:
gleamDF.head()

,Region,Animal species,Production system,Commodity,Emission Intensity (kg CO2e per kg protein),Production (kg protein),Total GHG emissions (kg CO2e),Total CO2 emissions (kg CO2e),Total CH4 emissions (kg CO2e),Total N2O emissions (kg CO2e),...,"Feed: fertilizer & crop residues, N2O (kg CO2e)","Feed: applied & deposited manure, N2O (kg CO2e)","LUC: soy & palm, CO2 (kg CO2e)","LUC: pasture expansion, CO2 (kg CO2e)","Enteric fermentation, CH4 (kg CO2e)","Manure management, CH4 (kg CO2e)","Manure management, N2O (kg CO2e)","Direct energy, CO2 (kg CO2e)","Indirect energy, CO2 (kg CO2e)","Postfarm, CO2 (kg CO2e)"
0,Global,Cattle,Aggregated,Aggregated,160.3,2.916310e+10,4.674630e+12,9.449299e+11,2.648727e+12,1.080973e+12,...,1.740732e+11,7.419864e+11,4.687954e+10,3.870062e+11,2.509275e+12,1.394522e+11,1.649130e+11,4.215542e+10,1.585653e+10,9.955932e+10
1,Global,Cattle,Aggregated,Milk,86.7,1.888089e+10,1.637519e+12,2.756914e+11,9.676442e+11,3.941834e+11,...,7.166208e+10,2.543885e+11,1.497421e+10,NaN,8.856352e+11,8.200906e+10,6.813278e+10,2.993485e+10,3.252889e+09,8.929544e+10
2,Global,Cattle,Aggregated,Meat,295.4,1.028222e+10,3.037111e+12,6.692385e+11,1.681083e+12,6.867892e+11,...,1.024111e+11,4.875979e+11,3.190534e+10,3.870062e+11,1.623640e+12,5.744310e+10,9.678019e+10,1.222057e+10,1.260364e+10,1.026388e+10
3,Global,Cattle,Grassland systems,Aggregated,206.3,1.033817e+10,2.133054e+12,5.628062e+11,1.052965e+12,5.172829e+11,...,5.253660e+10,4.267612e+11,8.194330e+09,3.870062e+11,9.969096e+11,5.605552e+10,3.798508e+10,1.417326e+10,4.303272e+09,3.773092e+10
4,Global,Cattle,Grassland systems,Milk,95.0,6.940654e+09,6.592844e+11,9.286140e+10,3.713461e+11,1.950770e+11,...,2.156126e+10,1.587838e+11,2.412100e+09,NaN,3.369018e+11,3.444428e+10,1.473188e+10,1.078605e+10,9.998435e+08,3.462338e+10


In [13]:
# filtering production system to only aggregated
gleamDF = gleamDF[gleamDF['Production system']=='Aggregated']
len(gleamDF)

176

In [14]:
# filtering commodity to only aggregated
### first gotta fix up pigs since they only have meat
gleamDF.loc[gleamDF['Animal species'] == 'Pigs', 'Commodity'] = 'Aggregated'
gleamDF = gleamDF[gleamDF['Commodity']=='Aggregated']
len(gleamDF)

66

In [15]:
# filtering region to only NA, EA/SEA, and SA 
# and animals to just cattle, chickens, and pigs

places = ['North America', 'East Asia and Southeast Asia', 'South Asia']
animals = ['Cattle', 'Pigs', 'Chicken']

gleamDF = gleamDF[gleamDF['Region'].isin(places)]
gleamDF = gleamDF[gleamDF['Animal species'].isin(animals)]
gleamDF

,Region,Animal species,Production system,Commodity,Emission Intensity (kg CO2e per kg protein),Production (kg protein),Total GHG emissions (kg CO2e),Total CO2 emissions (kg CO2e),Total CH4 emissions (kg CO2e),Total N2O emissions (kg CO2e),...,"Feed: fertilizer & crop residues, N2O (kg CO2e)","Feed: applied & deposited manure, N2O (kg CO2e)","LUC: soy & palm, CO2 (kg CO2e)","LUC: pasture expansion, CO2 (kg CO2e)","Enteric fermentation, CH4 (kg CO2e)","Manure management, CH4 (kg CO2e)","Manure management, N2O (kg CO2e)","Direct energy, CO2 (kg CO2e)","Indirect energy, CO2 (kg CO2e)","Postfarm, CO2 (kg CO2e)"
53,East Asia and Southeast Asia,Cattle,Aggregated,Aggregated,184.4,2.987340e+09,5.508116e+11,9.177232e+10,3.340070e+11,1.250323e+11,...,3.770133e+10,6.348248e+10,1.940060e+09,NaN,3.251330e+11,8.874031e+09,2.384849e+10,3.757554e+09,1.647682e+09,9.080869e+09
91,East Asia and Southeast Asia,Pigs,Aggregated,Aggregated,62.8,7.291823e+09,4.578821e+11,1.948683e+11,1.477716e+11,1.152422e+11,...,6.869189e+10,1.569576e+10,2.760193e+10,NaN,1.665441e+10,9.991075e+10,3.085459e+10,6.775204e+09,7.125333e+08,1.767153e+10
95,East Asia and Southeast Asia,Chicken,Aggregated,Aggregated,44.9,8.309233e+09,3.730866e+11,2.561117e+11,8.618132e+09,1.083568e+11,...,6.876894e+10,2.373567e+10,6.889729e+10,NaN,NaN,6.193934e+09,1.585217e+10,2.080202e+10,1.056223e+09,2.276913e+10
265,North America,Cattle,Aggregated,Aggregated,94.6,4.946003e+09,4.677560e+11,7.051503e+10,2.927900e+11,1.044510e+11,...,1.664071e+10,6.327627e+10,1.553472e+08,NaN,2.375667e+11,5.522328e+10,2.453400e+10,1.119346e+10,4.342067e+09,3.123173e+10
303,North America,Pigs,Aggregated,Aggregated,36.5,1.742059e+09,6.362182e+10,2.655649e+10,2.661064e+10,1.045469e+10,...,5.787965e+09,2.166712e+09,1.754332e+08,NaN,2.547485e+09,2.406315e+10,2.500017e+09,3.508714e+09,1.018953e+09,6.654234e+09
307,North America,Chicken,Aggregated,Aggregated,20.2,3.367069e+09,6.804370e+10,4.325686e+10,3.520174e+09,2.126667e+10,...,7.995893e+09,8.858449e+09,3.297324e+08,NaN,NaN,3.520174e+09,4.412324e+09,9.602130e+09,1.878925e+09,8.740349e+09
424,South Asia,Cattle,Aggregated,Aggregated,221.1,3.312079e+09,7.321946e+11,1.110645e+11,4.681744e+11,1.529558e+11,...,4.273179e+10,7.155194e+10,1.492877e+09,NaN,4.573832e+11,1.079124e+10,3.867203e+10,3.618877e+09,4.043465e+08,1.269176e+10
462,South Asia,Pigs,Aggregated,Aggregated,75.9,1.231999e+08,9.352089e+09,2.648732e+09,4.693916e+09,2.009440e+09,...,8.749754e+08,3.603544e+08,3.104926e+08,NaN,3.263805e+08,3.393300e+09,7.741106e+08,5.696733e+07,7.850124e+06,1.842217e+08
466,South Asia,Chicken,Aggregated,Aggregated,32.9,2.161700e+09,7.104950e+10,4.373422e+10,2.239874e+09,2.507541e+10,...,1.105900e+10,8.671136e+09,5.016711e+09,NaN,NaN,1.323886e+09,5.345273e+09,6.095856e+09,3.777266e+08,5.825515e+09


In [16]:
# shaving off unneeded columns:
gleamDF = gleamDF.iloc[:,:7]
gleamDF

,Region,Animal species,Production system,Commodity,Emission Intensity (kg CO2e per kg protein),Production (kg protein),Total GHG emissions (kg CO2e)
53,East Asia and Southeast Asia,Cattle,Aggregated,Aggregated,184.4,2.987340e+09,5.508116e+11
91,East Asia and Southeast Asia,Pigs,Aggregated,Aggregated,62.8,7.291823e+09,4.578821e+11
95,East Asia and Southeast Asia,Chicken,Aggregated,Aggregated,44.9,8.309233e+09,3.730866e+11
265,North America,Cattle,Aggregated,Aggregated,94.6,4.946003e+09,4.677560e+11
303,North America,Pigs,Aggregated,Aggregated,36.5,1.742059e+09,6.362182e+10
307,North America,Chicken,Aggregated,Aggregated,20.2,3.367069e+09,6.804370e+10
424,South Asia,Cattle,Aggregated,Aggregated,221.1,3.312079e+09,7.321946e+11
462,South Asia,Pigs,Aggregated,Aggregated,75.9,1.231999e+08,9.352089e+09
466,South Asia,Chicken,Aggregated,Aggregated,32.9,2.161700e+09,7.104950e+10


In [20]:
# Applying NA rate of emmision intensity to other regions' cattle production
cattleDF = gleamDF[gleamDF['Animal species']=='Cattle']
cattleDF['NA Rate'] = cattleDF['Production (kg protein)'] * 94.6
cattleDF['Saved Emmisions'] = cattleDF['Total GHG emissions (kg CO2e)'] - cattleDF['NA Rate']
cattleDF

<ipython-input-20-e8dbd4d9d898>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cattleDF['NA Rate'] = cattleDF['Production (kg protein)'] * 94.6
<ipython-input-20-e8dbd4d9d898>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cattleDF['Saved Emmisions'] = cattleDF['Total GHG emissions (kg CO2e)'] - cattleDF['NA Rate']


,Region,Animal species,Production system,Commodity,Emission Intensity (kg CO2e per kg protein),Production (kg protein),Total GHG emissions (kg CO2e),NA Rate,Saved Emmisions
53,East Asia and Southeast Asia,Cattle,Aggregated,Aggregated,184.4,2.987340e+09,5.508116e+11,2.826024e+11,2.682093e+11
265,North America,Cattle,Aggregated,Aggregated,94.6,4.946003e+09,4.677560e+11,4.678919e+11,-1.358985e+08
424,South Asia,Cattle,Aggregated,Aggregated,221.1,3.312079e+09,7.321946e+11,3.133227e+11,4.188720e+11


In [21]:
# Applying NA rate of emmision intensity to other regions' pigs production
pigsDF = gleamDF[gleamDF['Animal species']=='Pigs']
pigsDF['NA Rate'] = pigsDF['Production (kg protein)'] * 36.5
pigsDF['Saved Emmisions'] = pigsDF['Total GHG emissions (kg CO2e)'] - pigsDF['NA Rate']
pigsDF

<ipython-input-21-6a240c14455a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pigsDF['NA Rate'] = pigsDF['Production (kg protein)'] * 36.5
<ipython-input-21-6a240c14455a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pigsDF['Saved Emmisions'] = pigsDF['Total GHG emissions (kg CO2e)'] - pigsDF['NA Rate']


,Region,Animal species,Production system,Commodity,Emission Intensity (kg CO2e per kg protein),Production (kg protein),Total GHG emissions (kg CO2e),NA Rate,Saved Emmisions
91,East Asia and Southeast Asia,Pigs,Aggregated,Aggregated,62.8,7.291823e+09,4.578821e+11,2.661515e+11,1.917306e+11
303,North America,Pigs,Aggregated,Aggregated,36.5,1.742059e+09,6.362182e+10,6.358516e+10,3.666203e+07
462,South Asia,Pigs,Aggregated,Aggregated,75.9,1.231999e+08,9.352089e+09,4.496796e+09,4.855293e+09


In [22]:
# Applying NA rate of emmision intensity to other regions' chicken production
chickenDF = gleamDF[gleamDF['Animal species']=='Chicken']
chickenDF['NA Rate'] = chickenDF['Production (kg protein)'] * 20.2
chickenDF['Saved Emmisions'] = chickenDF['Total GHG emissions (kg CO2e)'] - chickenDF['NA Rate']
chickenDF

<ipython-input-22-da7e17746b5d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chickenDF['NA Rate'] = chickenDF['Production (kg protein)'] * 20.2
<ipython-input-22-da7e17746b5d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chickenDF['Saved Emmisions'] = chickenDF['Total GHG emissions (kg CO2e)'] - chickenDF['NA Rate']


,Region,Animal species,Production system,Commodity,Emission Intensity (kg CO2e per kg protein),Production (kg protein),Total GHG emissions (kg CO2e),NA Rate,Saved Emmisions
95,East Asia and Southeast Asia,Chicken,Aggregated,Aggregated,44.9,8.309233e+09,3.730866e+11,1.678465e+11,2.052401e+11
307,North America,Chicken,Aggregated,Aggregated,20.2,3.367069e+09,6.804370e+10,6.801479e+10,2.891430e+07
466,South Asia,Chicken,Aggregated,Aggregated,32.9,2.161700e+09,7.104950e+10,4.366634e+10,2.738316e+10


In [23]:
# Concatenating animals back together into one dataset:
combinedDF = pd.concat([cattleDF, pigsDF, chickenDF], ignore_index=True)
combinedDF

,Region,Animal species,Production system,Commodity,Emission Intensity (kg CO2e per kg protein),Production (kg protein),Total GHG emissions (kg CO2e),NA Rate,Saved Emmisions
0,East Asia and Southeast Asia,Cattle,Aggregated,Aggregated,184.4,2.987340e+09,5.508116e+11,2.826024e+11,2.682093e+11
1,North America,Cattle,Aggregated,Aggregated,94.6,4.946003e+09,4.677560e+11,4.678919e+11,-1.358985e+08
2,South Asia,Cattle,Aggregated,Aggregated,221.1,3.312079e+09,7.321946e+11,3.133227e+11,4.188720e+11
3,East Asia and Southeast Asia,Pigs,Aggregated,Aggregated,62.8,7.291823e+09,4.578821e+11,2.661515e+11,1.917306e+11
4,North America,Pigs,Aggregated,Aggregated,36.5,1.742059e+09,6.362182e+10,6.358516e+10,3.666203e+07
5,South Asia,Pigs,Aggregated,Aggregated,75.9,1.231999e+08,9.352089e+09,4.496796e+09,4.855293e+09
6,East Asia and Southeast Asia,Chicken,Aggregated,Aggregated,44.9,8.309233e+09,3.730866e+11,1.678465e+11,2.052401e+11
7,North America,Chicken,Aggregated,Aggregated,20.2,3.367069e+09,6.804370e+10,6.801479e+10,2.891430e+07
8,South Asia,Chicken,Aggregated,Aggregated,32.9,2.161700e+09,7.104950e+10,4.366634e+10,2.738316e+10
